In [129]:
%load_ext autoreload
%autoreload 2

from zipfile import ZipFile
from matplotlib import pyplot as plt 
from PIL import Image
import numpy as np
import sys

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

sys.path.insert(1, './../Aula02/')
from src.train import get_all_subsets


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [130]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cuda device


In [131]:
source_file = './dataset/brain-cancer.zip'

In [132]:
zip = ZipFile(source_file)

In [133]:
def get_label(filename):
    parts = filename.lower().split('/')
    result = None
    if 'yes' in parts:
        result = 1
    elif 'no' in parts:
        result = 0
    else:
        pass
    
    return result

def get_image_array(filename):
    new_size = (224, 224)
    img_data = Image.open(zip.open(filename))
    img_array = np.array(img_data.resize(new_size), np.int8)
    if img_array.ndim == 1: 
        # Replicar o canal para imagens em escala de cinza
        img_array = np.expand_dims(img_array, axis = -1)
        img_array = np.concatenate([img_array] * 3, axis = -1)

    # normalizar
    img_array_normalized = (img_array - img_array.mean()) / img_array.std()

    return img_array_normalized


assert get_label('brain_tumor_dataset/no/30 no.jpg') == 0
assert get_label('brain_tumor_dataset/NO/31 no.jpg') == 0
assert get_label('yes/Y95.jpg') == 1
assert get_label('brain_tumor_dataset/yes/Y155.JPG') == 1
assert get_label('brain_tumor_dataset/Y155.JPG') == None

In [134]:
# process
images = []
labels = []
data_results_dict = []

for filename in zip.namelist():
    
    # get label 0 = no, 1 = yes
    label = get_label(filename)

    # get image array    
    img_array = get_image_array(filename) 
    
    # build a dict with data
    data_results_dict.append(
        {'filename': filename, 
         'label': label, 
         'img_array': img_array}
    )

    if img_array.shape == (224, 224, 3):
        # get X and y vars
        images.append(img_array)
        labels.append(label)


In [135]:
len(images), len(labels)

(434, 434)

In [136]:
# all images have 3 channels
size = []
for a in images:
    size.append(len(a.shape))

np.unique(size, return_counts=True)

(array([3]), array([434]))

In [137]:
# y is unbalanced
np.unique(labels, return_counts=True)

(array([0, 1]), array([158, 276]))

In [138]:
# getting 
X_train, X_val, X_test, y_train, y_val, y_test = get_all_subsets(np.array(images), 
                                                                 np.array(labels))

In [142]:
print("Shape X_train: ", X_train.shape)
print("Shape X_val: ", X_val.shape)
print("Shape X_test: ", X_test.shape)

print("Shape y_train: ", y_train.shape)
print("Shape y_val: ", y_val.shape)
print("Shape y_test: ", y_test.shape)


Shape X_train:  (277, 224, 224, 3)
Shape X_val:  (70, 224, 224, 3)
Shape X_test:  (87, 224, 224, 3)
Shape y_train:  (277,)
Shape y_val:  (70,)
Shape y_test:  (87,)


In [71]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.flatten = nn.Flatten()
        
        self.history = nn.Sequential()
        self.history.add_module(self.flatten)
        self.history.add_module(
        #     nn.Linear(28*28, 512),
        #     nn.ReLU(),
        #     nn.Linear(512, 512),
        #     nn.ReLU(),
        #     nn.Linear(512, 10),
        # )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.history(x)
        return logits

SyntaxError: '(' was never closed (1963034406.py, line 8)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
model = CNNModel().to(device)
model

CNNModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
logits = model(X_train)

AttributeError: 'list' object has no attribute 'flatten'

In [79]:
ones = np.ones((10,10, 3))

ones



array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
  